In [1]:
pip install -U git+https://github.com/jlcsilva/segmentation_models.pytorch


  Cloning https://github.com/jlcsilva/segmentation_models.pytorch to /tmp/pip-req-build-znzwzq2m
  Running command git clone --filter=blob:none --quiet https://github.com/jlcsilva/segmentation_models.pytorch /tmp/pip-req-build-znzwzq2m
  Resolved https://github.com/jlcsilva/segmentation_models.pytorch to commit 44a5593bfb1c8f2a3f5f23303c02e6f7e2f11170
  Preparing metadata (setup.py) ... done
  Using cached efficientnet_pytorch-0.6.3.tar.gz (16 kB)
  Preparing metadata (setup.py) ... done
  Using cached timm-0.4.12-py3-none-any.whl (376 kB)
Using legacy 'setup.py install' for segmentation-models-pytorch, since package 'wheel' is not installed.
Using legacy 'setup.py install' for efficientnet-pytorch, since package 'wheel' is not installed.
  Attempting uninstall: efficientnet-pytorch
    Found existing installation: efficientnet-pytorch 0.7.1
    Uninstalling efficientnet-pytorch-0.7.1:
      Successfully uninstalled efficientnet-pytorch-0.7.1
  Running setup.py install for efficientnet

/media/gaston/gaston1/pro/envs/actia/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/media/gaston/gaston1/pro/envs/actia/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import torch.nn as nn
import segmentation_models_pytorch as smp
from torchinfo import summary
class Res50Unet(nn.Module):
    def __init__(self, num_classes, in_channels=3,start_filts=64, depth=4):
        super().__init__()

        self.num_classes = num_classes
        self.in_channels = in_channels
        self.depth = depth
        self.start_filts=start_filts 

        decoder_channels=tuple([(2**i)*self.start_filts for i in range(self.depth,0,-1)])
        self.model =  smp.UnetPlusPlus(
            encoder_name='resnet50', 
            encoder_depth=self.depth, 
            encoder_weights='imagenet', 
            decoder_use_batchnorm=True, 
            decoder_channels=decoder_channels, 
            decoder_attention_type=None, 
            in_channels=self.in_channels, 
            classes=self.num_classes, 
            activation=None,
            aux_params=None
            )

    def forward(self, inputs):
        return self.model(inputs)
    
    def to(self,device):
        self.model=self.model.to(device)
        return super().to(device)
    
model0=Res50Unet(19,3,64,4)

for param in model0.model.encoder.parameters():
    param.requires_grad = False

summary(model0, input_size=(1, 3,64,64))


Layer (type:depth-idx)                             Output Shape              Param #
Res50Unet                                          [1, 19, 64, 64]           --
├─UnetPlusPlus: 1-1                                [1, 19, 64, 64]           --
│    └─ResNetEncoder: 2-1                          [1, 3, 64, 64]            14,964,736
│    │    └─Conv2d: 3-1                            [1, 64, 32, 32]           (9,408)
│    │    └─BatchNorm2d: 3-2                       [1, 64, 32, 32]           (128)
│    │    └─ReLU: 3-3                              [1, 64, 32, 32]           --
│    │    └─MaxPool2d: 3-4                         [1, 64, 16, 16]           --
│    │    └─Sequential: 3-5                        [1, 256, 16, 16]          (215,808)
│    │    └─Sequential: 3-6                        [1, 512, 8, 8]            (1,219,584)
│    │    └─Sequential: 3-7                        [1, 1024, 4, 4]           (7,098,368)
│    └─UnetPlusPlusDecoder: 2-2                    [1, 128, 64, 64]       

In [2]:
import wandb
import torch
import torch.nn as nn
import torch.optim as optim

# Initialize a new wandb run
wandb.init(project="your_project_name")

# Example model
model =model0

# Log the model graph to wandb
wandb.watch(model, log="all")

# Dummy data
inputs = torch.zeros(1, 1, 16, 16)
targets = torch.zeros(1, 19, 16, 16)  # Adjust to match output shape

# Define a simple loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Forward pass
outputs = model(inputs)
loss = criterion(outputs, targets)

# Backward pass and optimization
optimizer.zero_grad()
loss.backward()
optimizer.step()

# Log metrics and save model
wandb.log({"loss": loss.item()})
wandb.save('model.pth')

# Finish the wandb run
wandb.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

In [ ]:

model6=Res50UnetPP(num_classes=19,in_channels=3,start_filts=32,depth=4)
# Print model summary using torchinfo
summary(model6, input_size=(1, 3,256,256), device=device)